In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extraction with Tweepy (limited amount of tweets and date range)

We started using Tweepy but it turned to be not a good option due to that it is limited. Therefore we stop with Tweepy and used snscrape.

In [ ]:
import tweepy

# replace these values with yours
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth)

In [ ]:
#### Define twitter rate determining loop
def twitter_rates():
    stats = api.rate_limit_status()  #stats['resources'].keys()
    for akey in stats['resources'].keys():
        if type(stats['resources'][akey]) == dict:
            for anotherkey in stats['resources'][akey].keys():
                if type(stats['resources'][akey][anotherkey]) == dict:
                    #print(akey, anotherkey, stats['resources'][akey][anotherkey])
                    limit = (stats['resources'][akey][anotherkey]['limit'])
                    remaining = (stats['resources'][akey][anotherkey]['remaining'])
                    used = limit - remaining
                    if used != 0:
                        print("Twitter API used", used, "remaining queries", remaining,"for query type", anotherkey)
                    else:
                        pass
                else:
                    pass  #print("Passing")  #stats['resources'][akey]
        else:
            print(akey, stats['resources'][akey])
            print(stats['resources'][akey].keys())
            limit = (stats['resources'][akey]['limit'])
            remaining = (stats['resources'][akey]['remaining'])
            used = limit - remaining
            if used != 0:
                print("Twitter API:", used, "requests used,", remaining, "remaining, for API queries to", akey)
                pass


twitter_rates()

Twitter API used 1 remaining queries 179 for query type /application/rate_limit_status
Twitter API used 75 remaining queries 105 for query type /search/tweets


## Get twitter data

By user

In [ ]:
tweets = api.user_timeline(id='nytimes', count=20)

for tweet in tweets:
 print(tweet.text)

“The panic, the anxious feelings, it has all been too much,” said one Michigan voter, who like many Americans has t… https://t.co/da2JBUuhz9
Watch just how effective a mask can be at helping to stop the spread of the coronavirus and protecting you and othe… https://t.co/sjguVXK2c1
The CDC now says that wearing a mask not only helps protect those around you from the coronavirus, but it may also… https://t.co/gDhSIc0dgQ
It’s open enrollment season for the Affordable Care Act. If you need health insurance, here’s how to sign up. https://t.co/GGor3CotJc
Britney Spears’s life has been controlled by a court-approved conservatorship, a legal arrangement designed for peo… https://t.co/gsxiAt5x88
President Trump and his allies' claims of rampant voter fraud haven't targeted every state. They've been noticeably… https://t.co/waSVfYhDXy
On Saturday, Rudy Giuliani, the president’s personal lawyer, claimed without evidence that the election in Philadel… https://t.co/9JeESQHXQq
In Michigan, the Trum

### By word

In [ ]:
import time
import datetime


query = "lufthansa"
language = "en"
BOLD = "\033[1m"
NORMAL = "\033[0m"

# tweets = api.search(q=query, lang=language, tweet_mode='extended', count=1000)

# for tweet in tweets:
#     print(BOLD + "User: " + NORMAL + tweet.user.screen_name + "\nTweet text: " + tweet.full_text + "\n")

# tweets_list = list()
# # above omitted for brevity
# c = tweepy.Cursor(api.search
#                   , q=query
#                   , lang = language
#                   , tweet_mode='extended'
#                   , include_entities=True
#                   ).items()

# while True and (tmpTweets[-1].created_at > startDate) and len(tweets_list)<1000:
#     try:
#         tweet = c.next()
#         if tweet.created_at < end_date and tweet.created_at > start_date:
#           tweets_list.append(tweet)
#         # print(tweet)
#         # Insert into db
#     except tweepy.TweepError:
#         print(len(tweets_list))
#         time.sleep(60 * 15)
#         continue
#     except StopIteration:
#         print(len(tweets_list)) 
#         break

startDate = datetime.datetime(2020, 1, 1)
endDate = datetime.datetime(2020, 1, 30)

tweets = []
tmpTweets = api.search(q=query, lang=language, tweet_mode='extended', max_id=1325955458485700998)
for tweet in tmpTweets:
    if tweet.created_at < endDate and tweet.created_at > startDate:
        tweets.append(tweet)

while (tmpTweets[-1].created_at > startDate):
    print("Last Tweet @", tmpTweets[-1].created_at, " - fetching some more")
    tmpTweets = api.search(q=query, lang=language, tweet_mode='extended', max_id = tmpTweets[-1].id)
    print((tmpTweets[-1].id))
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)


print(len(tweets_list))
# print(tweets_list)

Last Tweet @ 2020-11-09 22:56:17  - fetching some more
1325922020828340225
Last Tweet @ 2020-11-09 22:03:38  - fetching some more
1325911945938153474
Last Tweet @ 2020-11-09 21:23:36  - fetching some more
1325899548896583680
Last Tweet @ 2020-11-09 20:34:21  - fetching some more
1325893591403728897
Last Tweet @ 2020-11-09 20:10:40  - fetching some more
1325886295500009473
Last Tweet @ 2020-11-09 19:41:41  - fetching some more
1325880835933921281
Last Tweet @ 2020-11-09 19:19:59  - fetching some more
1325870704718475273
Last Tweet @ 2020-11-09 18:39:44  - fetching some more
1325852625200406534
Last Tweet @ 2020-11-09 17:27:53  - fetching some more
1325839921928671237
Last Tweet @ 2020-11-09 16:37:24  - fetching some more
1325838524592099330
Last Tweet @ 2020-11-09 16:31:51  - fetching some more
1325834873035235329
Last Tweet @ 2020-11-09 16:17:21  - fetching some more
1325829316278620161
Last Tweet @ 2020-11-09 15:55:16  - fetching some more
1325826321277210626
Last Tweet @ 2020-11-09 1

KeyboardInterrupt: ignored

# Extraction with snscrape (unlimited)

As we encountered limitations when using Tweepy, snscrape has been chosen to download twitter's data.

We have downloaded monthly data for the top 10 airlines in Europe during 2020, setting the maximum number of tweets for each period to be 10000.


In [2]:
!pip install snscrape

In [7]:
import snscrape.modules.twitter as sntwitter
import csv

dir = "/content/drive/My Drive/Project/Tweets_data/"


keywords_list = [
                 '@lufthansa', '@British_Airways', '@ryanair', '@iberia', '@SAS'
                , '@vueling', '@airfrance', '@klm', '@easyjet', '@Fly_Norwegian'
                ]
months = range(1, 11)
maxTweets = 10000

for keyword in keywords_list:
  for month in months:
    start_month = month
    end_month = month + 2
    #Open/create a file to append data to
    filename = dir + keyword + "_M_" + str(month) + '.csv'
    #csv_file = open(filename, 'a', newline='', encoding='utf8')

    #Use csv writer
    #csv_writer = csv.writer(csv_file)
    #csv_writer.writerow(['id','date','tweet']) 
    
    if month ==2:
        last_day = 28
    elif month in [1,3,5,7,8,10,12]:
        last_day = 31
    else:
        last_day = 30

    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' since:2020-' + str(month) + '-01 until:2020-' + str(month) + '-'+str(last_day)+'- lang:en').get_items()):
      # print(i, tweet.date, tweet, tweet.content)
      if i > maxTweets:
        break
       #print(i, tweet.id, tweet.date, tweet.content)
      #csv_writer.writerow([tweet.id, tweet.date, tweet.content])
    #csv_file.close()

    print("Tweets stored for keyword {} and month m{}-2020".format(keyword, month))
    if month == 2:
      break
  break

Tweets stored for keyword @lufthansa and month m1-2020
Tweets stored for keyword @lufthansa and month m2-2020
